In [16]:
class Rotor:
    def __init__(self, wiring, notch, position=0):
        # wiring: 26-char string mapping input to output
        # notch: position at which next rotor is advanced
        # position: initial rotor position (0-25)
        self.wiring = wiring
        self.notch = notch
        self.position = position
        self.ring_setting = 0  # For simplicity, set ring setting to 0

    def encode_forward(self, c):
        # c: int 0-25 input letter index
        offset_c = (c + self.position - self.ring_setting) % 26
        wired_c = ord(self.wiring[offset_c]) - ord('A')
        output = (wired_c - self.position + self.ring_setting) % 26
        return output

    def encode_backward(self, c):
        offset_c = (c + self.position - self.ring_setting) % 26
        wired_c = self.wiring.index(chr(offset_c + ord('A')))
        output = (wired_c - self.position + self.ring_setting) % 26
        return output

    def step(self):
        self.position = (self.position + 1) % 26
        # Returns True if rotor hits notch and should cause next rotor to step
        return chr((self.position + ord('A')) % 26) == self.notch

class Reflector:
    def __init__(self, wiring):
        self.wiring = wiring

    def reflect(self, c):
        return ord(self.wiring[c]) - ord('A')

class EnigmaMachine:
    def __init__(self, rotors, reflector):
        # rotors: list of Rotor objects from right to left
        self.rotors = rotors
        self.reflector = reflector

    def step_rotors(self):
        # Step rotors like odometer with double stepping
        step_next = self.rotors[0].step()
        # For double stepping, check second rotor if first rotor is at notch
        if step_next:
            step_next_2 = self.rotors[1].step() if len(self.rotors) > 1 else False
            if step_next_2 and len(self.rotors) > 2:
                self.rotors[2].step()

    def encode_char(self, ch):
        if not ch.isalpha():
            return ch
        c = ord(ch.upper()) - ord('A')

        self.step_rotors()

        # Pass through rotors forward
        for rotor in self.rotors:
            c = rotor.encode_forward(c)

        # Reflect
        c = self.reflector.reflect(c)

        # Pass through rotors backward (reverse order)
        for rotor in reversed(self.rotors):
            c = rotor.encode_backward(c)

        return chr(c + ord('A'))

    def encode_message(self, message):
        result = ""
        for ch in message:
            result += self.encode_char(ch)
        return result


In [17]:

# Example configurations:

# Rotor wirings taken from historical Enigma I
ROTOR_I = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
ROTOR_II = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
ROTOR_III = "BDFHJLCPRTXVZNYEIWGAKMUSQO"

# Notch positions
NOTCH_I = 'Q'  # When rotor steps from Q to R, next rotor steps
NOTCH_II = 'E'
NOTCH_III = 'V'

# Reflector B wiring
REFLECTOR_B = "YRUHQSLDPXNGOKMIEBFZCWVJAT"

# Setup rotors with initial positions
rotor1 = Rotor(ROTOR_I, NOTCH_I, position=0)    # Right rotor
rotor2 = Rotor(ROTOR_II, NOTCH_II, position=0)  # Middle rotor
rotor3 = Rotor(ROTOR_III, NOTCH_III, position=0) # Left rotor

reflector = Reflector(REFLECTOR_B)

# Create machine
enigma = EnigmaMachine([rotor1, rotor2, rotor3], reflector)

input_text = "HELLOWORLD"
print("Input:       ", input_text)
encoded = enigma.encode_message(input_text)
print("Encoded:     ", encoded)

# To decode, reset rotors to same initial positions and use the same machine
rotor1 = Rotor(ROTOR_I, NOTCH_I, position=0)
rotor2 = Rotor(ROTOR_II, NOTCH_II, position=0)
rotor3 = Rotor(ROTOR_III, NOTCH_III, position=0)
enigma = EnigmaMachine([rotor1, rotor2, rotor3], reflector)

decoded = enigma.encode_message(encoded)
print("Decoded:     ", decoded)


Input:        HELLOWORLD
Encoded:      MFNCZBBFZM
Decoded:      HELLOWORLD
